In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# .env 파일 로드
load_dotenv()

# langSmith에 로깅 할 프로젝트 명을 입력
logging.langsmith("LANGCHAIN-CHOON")
# logging.langsmith("LANGCHAIN-CHOON", set_enable=False)      # LangSmith 추적 비활성화

LangSmith 추적을 시작합니다.
[프로젝트명]
LANGCHAIN-CHOON


## 1. 프롬프트 템플릿의 활용

langchain에서 제공하는 프롬프트 템플릿을 이용하여 사용자 입력 변수에 대응하기 위한 프롬프트 템플릿을 만들 수 있다


In [3]:
from langchain_core.prompts import PromptTemplate

`from_template()` 메서드를 사용하여 PromptTemplate 객체 생성

In [4]:
# template 정의
template = "{city}의 NFL 팀 이름은 무엇인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['city'], input_types={}, partial_variables={}, template='{city}의 NFL 팀 이름은 무엇인가요?')

In [8]:
# Prompt 생성
prompt = prompt_template.format(city="미네소타")
prompt

'미네소타의 NFL 팀 이름은 무엇인가요?'

In [6]:
# Prompt 생성
prompt = prompt_template.format(city="LA")
prompt

'LA의 NFL 팀 이름은 무엇인가요?'

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

print(llm.invoke(prompt))

content='미네소타의 NFL 팀 이름은 미네소타 바이킹스(Minnesota Vikings)입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 20, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None} id='run-b8aa4474-2745-4a27-b2de-b5c605736010-0' usage_metadata={'input_tokens': 20, 'output_tokens': 24, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## LCEL (LangChain Expression Language)

LCEL을 사용하여 다양한 구성 요소를 하나의 체인으로 결합한다

chain = prompt | model 혹은
chain = prompt | model | model 등등 다양하게 본인만의 형태로 chain을 구성할 수 있다

요소끼리 결합하는 방법은 요소 사이에 '|' 를 넣어주면 된다.
요소끼리 결합을 하게 되면 prompt 요소에서 발생한 결과가 다음 요소인 model에게로 전달된다.


In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# prompt를 PromptTemplate 객체로 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4o", temperature=0.3)

# prompt 와 model을 결합하여 하나의 체인으로 생성
chain = prompt | model

### invoke() 호출

- invoke() 메서드 호출 시, 입력값을 전달

In [12]:
input = {"topic": "NBA"}

In [13]:
# prompt 객체와 model 객체를 | 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
chain.invoke(input)

AIMessage(content='NBA는 "National Basketball Association"의 약자로, 미국의 프로 농구 리그입니다. NBA는 세계에서 가장 유명하고 수준 높은 농구 리그 중 하나로, 많은 팬들이 경기를 즐기고 있습니다. \n\nNBA는 1946년에 설립되었으며, 현재 30개의 팀이 소속되어 있습니다. 이 팀들은 미국과 캐나다에 위치해 있으며, 동부 콘퍼런스와 서부 콘퍼런스로 나뉘어 있습니다. 각 콘퍼런스는 다시 3개의 디비전으로 나뉩니다.\n\nNBA 시즌은 보통 10월에 시작하여 다음 해 4월까지 진행되며, 각 팀은 정규 시즌 동안 82경기를 치릅니다. 정규 시즌이 끝나면 각 콘퍼런스에서 상위 8개 팀이 플레이오프에 진출하여 최종적으로 NBA 챔피언을 가리게 됩니다. 플레이오프는 7전 4선승제로 진행됩니다.\n\nNBA는 마이클 조던, 르브론 제임스, 코비 브라이언트 등 전설적인 선수들을 배출했으며, 이들은 농구의 인기를 전 세계적으로 확산시키는 데 큰 역할을 했습니다. NBA는 또한 다양한 문화적, 사회적 이슈에 대한 목소리를 내며 스포츠 이상의 영향력을 발휘하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 296, 'prompt_tokens': 14, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish

### 출력 파서 (Output Parser)

- 결과 출력시 결과 내용만 출력하고자하 할때 출력 파서를 체인에 추가한다

In [19]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

input = {"topic": "NFL"}

# 프롬프트 , 모델, 출력 파서를 연결하여 하나의 체인으로 구성
chain = prompt | model | output_parser

In [15]:
print(chain.invoke(input))

NFL은 "National Football League"의 약자로, 미국의 프로 미식축구 리그입니다. NFL은 미국에서 가장 인기 있는 스포츠 리그 중 하나로, 32개의 팀이 두 개의 컨퍼런스(아메리칸 풋볼 컨퍼런스와 내셔널 풋볼 컨퍼런스)로 나뉘어 있습니다. 각 팀은 정규 시즌 동안 17경기를 치르며, 성적에 따라 플레이오프에 진출하게 됩니다.

플레이오프는 단판 승부로 진행되며, 최종적으로 두 컨퍼런스의 챔피언 팀이 슈퍼볼에서 맞붙어 리그의 최종 우승자를 가립니다. 슈퍼볼은 전 세계적으로 큰 관심을 받는 스포츠 이벤트 중 하나로, 많은 사람들이 경기를 시청하고 다양한 광고와 공연이 함께 진행됩니다.

NFL 경기는 총 4쿼터로 나뉘어 있으며, 각 쿼터는 15분입니다. 팀은 공격과 수비를 번갈아 가며 경기를 진행하며, 상대 팀의 엔드존에 공을 가지고 들어가거나 필드골을 성공시켜 점수를 얻습니다. NFL은 전략과 체력이 중요한 스포츠로, 다양한 전술과 선수들의 뛰어난 기량이 경기를 흥미롭게 만듭니다.


In [21]:
# 스트리밍 출력
stream_answer = chain.stream(input)
for token in stream_answer:
    print(token, end="", flush=True)

NFL은 "National Football League"의 약자로, 미국의 프로 미식축구 리그입니다. NFL은 미국에서 가장 인기 있는 스포츠 리그 중 하나로, 32개의 팀이 두 개의 컨퍼런스(AFC와 NFC)로 나뉘어 경기를 펼칩니다. 시즌은 보통 9월에 시작하여 12월 말까지 정규 시즌 경기가 진행되며, 각 팀은 17경기를 치릅니다.

정규 시즌이 끝나면 각 컨퍼런스에서 상위 팀들이 플레이오프에 진출하여, 최종적으로 두 컨퍼런스의 챔피언이 슈퍼볼(Super Bowl)에서 맞붙게 됩니다. 슈퍼볼은 미국에서 가장 큰 스포츠 이벤트 중 하나로, 전 세계적으로 많은 사람들이 시청합니다. NFL은 뛰어난 선수들, 전략적인 경기 운영, 그리고 화려한 쇼맨십으로 유명하며, 미국 문화의 중요한 부분을 차지하고 있습니다.

### 템플릿을 변경하여 적용

- 템플릿을 응용하여 템플릿 내 작성한 FORMAT 형태로 답변을 받을 수 있다

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = """
 당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
 양식은 [FORMAT]을 참고하여 작성해 주세요.

 # 상황: {question}

 # FORMAT:
 - 영어 회화:
 - 한글 해석:
"""

prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [24]:
input = {"question": "스타벅스에서 음료를 주문하고 싶어요"}

print(chain.invoke(input))

- 영어 회화:
  - Barista: Hi, welcome to Starbucks! What can I get for you today?
  - Customer: Hi, I’d like to order a tall caramel macchiato, please.
  - Barista: Sure, would you like it hot or iced?
  - Customer: I’ll have it hot, please.
  - Barista: Great! Anything else?
  - Customer: No, that’s all. Thank you.
  - Barista: Alright, your total is $4.75. Can I have your name for the order?
  - Customer: It’s Alex.
  - Barista: Thank you, Alex. Your order will be ready shortly.

- 한글 해석:
  - 바리스타: 안녕하세요, 스타벅스에 오신 것을 환영합니다! 오늘 무엇을 도와드릴까요?
  - 손님: 안녕하세요, 톨 사이즈의 카라멜 마키아또 하나 주문하고 싶어요.
  - 바리스타: 네, 따뜻하게 드릴까요 아니면 차갑게 드릴까요?
  - 손님: 따뜻하게 주세요.
  - 바리스타: 알겠습니다! 다른 건 필요 없으세요?
  - 손님: 아니요, 그게 전부입니다. 감사합니다.
  - 바리스타: 알겠습니다, 총 금액은 $4.75입니다. 주문하신 분 이름을 말씀해 주시겠어요?
  - 손님: 알렉스입니다.
  - 바리스타: 감사합니다, 알렉스님. 주문하신 음료는 곧 준비됩니다.


In [ ]:
input = {"question": "치폴레에서 주문을 하고 싶어요"}

answer_stream = chain.stream(input)

for token in answer_stream:
    print(token, end="")

### Batch : 배치 (단위 실행)

batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아 일괄 처리를 수행

In [30]:
input = [
    {
        "question": "레스토랑에서 와인을 주문하고 싶어요",
        "question": "맥도날드에서 빅맥을 주문하고 싶어",
    }
]

print(chain.batch(input))

["- 영어 회화:\n  - Customer: Hi, I'd like to order a Big Mac, please.\n  - Cashier: Sure! Would you like to make it a meal with fries and a drink?\n  - Customer: Yes, please. I'll have a medium meal.\n  - Cashier: Great. What drink would you like with that?\n  - Customer: I'll have a Coke, please.\n  - Cashier: Perfect. Anything else for you today?\n  - Customer: No, that's all. Thank you.\n  - Cashier: Your total is $XX.XX. Please drive to the next window.\n  - Customer: Thank you!\n\n- 한글 해석:\n  - 고객: 안녕하세요, 빅맥 하나 주문하고 싶어요.\n  - 캐셔: 네! 감자튀김과 음료가 포함된 세트로 드릴까요?\n  - 고객: 네, 중간 사이즈 세트로 할게요.\n  - 캐셔: 좋습니다. 어떤 음료로 드릴까요?\n  - 고객: 콜라로 주세요.\n  - 캐셔: 완벽합니다. 오늘은 이게 전부인가요?\n  - 고객: 네, 이게 다예요. 감사합니다.\n  - 캐셔: 총 금액은 $XX.XX입니다. 다음 창으로 이동해 주세요.\n  - 고객: 감사합니다!"]


max_concurrency 를 정의하면 한 번에 처리한 딕셔너리의 수를 정의할 수 있다

In [31]:
input = [
    {
        "question": "레스토랑에서 와인을 주문하고 싶어요",
        "question": "맥도날드에서 빅맥을 주문하고 싶어",
        "question": "대형마트에서 TV를 주문하고 싶어",
    }
]

print(chain.batch(input, config={"max_concurrency": 2}))

["- 영어 회화:\n  - Customer: Excuse me, could you help me with ordering a TV?\n  - Salesperson: Of course! Do you have a specific model in mind?\n  - Customer: Yes, I'm interested in the 55-inch Ultra HD model.\n  - Salesperson: Great choice. Let me check our stock. Would you like it delivered to your home?\n  - Customer: Yes, home delivery would be perfect. How long does it usually take?\n  - Salesperson: It typically takes 3 to 5 business days. Is that okay for you?\n  - Customer: Yes, that's fine. What are the payment options?\n  - Salesperson: You can pay by credit card, debit card, or cash. Which would you prefer?\n  - Customer: I'll pay with my credit card.\n  - Salesperson: Perfect. I will process your order right away. Can I have your delivery address, please?\n  - Customer: Sure, it's 123 Maple Street, Apartment 4B.\n  - Salesperson: Thank you. Your order is all set. Is there anything else I can assist you with?\n  - Customer: No, that's all. Thank you for your help!\n  - Salespe

## 2. 비동기 호출

### async stream : 비동기 스트림

주어진 토픽에 대해 메시지를 비동기적으로 처리

In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{topic} 에 대해 3문장으로 설명해줘.")

model = ChatOpenAI(model="gpt-4o")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

비동기 스트림은 astream 으로 호출
문자열 표시시 사용할 for 문 앞에 async를 붙여줘야 함

In [33]:
input = {"topic": "YouTube"}
async_stream = chain.astream(input)

async for token in async_stream:
    print(token, end="", flush=True)

YouTube는 사용자가 동영상을 업로드하고 시청할 수 있는 세계 최대의 비디오 공유 플랫폼입니다. 다양한 콘텐츠 제작자들이 엔터테인먼트, 교육, 리뷰 등 다양한 주제의 동영상을 제공하며, 사용자들은 이를 구독하고 댓글을 남기며 소통할 수 있습니다. 또한, YouTube는 광고를 통해 수익을 창출하고, 크리에이터들도 이를 통해 수익을 얻을 수 있는 파트너십 프로그램을 제공합니다.

### async invoke : 비동기 호출

ainvoke를 이용하여 호출
await 시 결과 호출

In [34]:
input = {"topic": "Apple"}

async_invoke = chain.ainvoke(input)
await async_invoke

'Apple은 1976년에 설립된 미국의 다국적 기술 기업으로, 주로 혁신적인 전자 기기와 소프트웨어를 개발 및 판매합니다. 대표적인 제품으로는 iPhone, iPad, MacBook 등이 있으며, iOS와 macOS 같은 운영체제를 통해 사용자 경험을 극대화하는 데 중점을 둡니다. 또한 Apple은 디자인, 보안, 생태계 통합에 대한 높은 기준을 유지하면서 전 세계적으로 강력한 브랜드 인지도를 가지고 있습니다.'

### async batch: 비동기 배치

abatch를 이용하여 호출
await 시 결과 호출

In [ ]:
input = [{"topic": "Sony"}, {"topic": "Palantir"}]

async_batch = chain.abatch(input)
await async_batch

['소니(SONY)는 일본에 본사를 둔 다국적 기업으로, 전자 제품, 엔터테인먼트 및 금융 서비스 분야에서 세계적으로 유명합니다. 이 회사는 텔레비전, 카메라, 오디오 기기와 같은 다양한 소비자 전자 제품을 제조하며, 플레이스테이션(PlayStation)과 같은 게임 콘솔로도 잘 알려져 있습니다. 또한, 소니는 영화, 음악 및 게임 콘텐츠 제작과 배급에서도 큰 영향을 미치고 있는 엔터테인먼트 강자입니다.',
 'Palantir는 주로 정부 기관과 대기업을 대상으로 데이터를 분석하고 관리하는 소프트웨어를 개발하는 미국의 기술 회사입니다. 이 회사는 복잡한 데이터 세트를 시각화하고 해석하는 데 도움을 주는 플랫폼을 제공하여 의사 결정 과정을 지원합니다. Palantir의 주요 제품 중에는 Palantir Gotham과 Palantir Foundry가 있으며, 각각 정보 분석과 데이터 통합을 위한 도구로 사용됩니다.']

## 3. Parallel: 병렬성

이전 과정에서 chain 은 chain = prompt | model | output_parser 이러한 형태로 구성했었는데 langchain에서는 prompt , model 각각을 Runnable 클래스로 이루어진 객체라 애기하며 이 RunnableParallel 클래스를 이용하면 Runnable 객체를 병렬로 실행할 수 있다.

In [37]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?") | model | output_parser
)

chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?") | model | output_parser
)

combined = RunnableParallel(capital=chain1, area=chain2)

In [38]:
combined.invoke({"country": "USA"})

{'capital': '미국의 수도는 워싱턴 D.C.입니다.',
 'area': '미국의 총 면적은 약 983만 평방킬로미터(약 379만 평방마일)입니다. 이 면적은 육지와 수역을 포함한 전체 면적을 나타냅니다.'}

In [40]:
combined.batch([{"country": "France"}, {"country": "Vietnam"}])

[{'capital': '프랑스의 수도는 파리입니다.',
  'area': '프랑스의 면적은 약 551,695 제곱킬로미터입니다. 이는 유럽 대륙에 위치한 본토 프랑스의 면적을 기준으로 한 것입니다. 프랑스는 또한 해외 영토를 포함하고 있으며, 이들까지 포함하면 총 면적은 더 커집니다.'},
 {'capital': '베트남의 수도는 하노이입니다.', 'area': '베트남의 면적은 약 331,212 제곱킬로미터입니다.'}]